In [117]:
import os
import pandas as pd
import logging

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Получаем путь к scrapped_videos_csv
current_directory = os.getcwd()

directory = current_directory + '\Scrapped_videos_csv'

## EDA

Итак, приступим к осмотру нашего датасета. В первую очередь поймем, какие столбцы нам точно не понадобятся в дальнейшем.  
Можно убрать video_id, video_title, channel_id, channel_name, channel_description, description, как незначительные признаки.

In [120]:
df = pd.read_csv(directory + '\\' + 'all_videos_data_no_duplicates.csv')
df_cleaned = df_no_duplicates.drop(columns=['video_id', 'video_title', 'channel_id', 'channel_name', 'channel_description', 'description'])

Далее посмотрим на пропуски в наших данных

In [121]:
df_cleaned.isna().sum()

published_at         0
views                0
likes             2139
comments          2035
category             5
subscribers        231
videos_count       231
source               0
comments_likes    6219
tags              7831
dtype: int64

У нас есть одинаковое кол-во пропусков в полях subscribers и videos_count. Посмотрим на эти строки

In [122]:
df_cleaned[(df_cleaned['subscribers'].isna()) & (df_cleaned['videos_count'].isna())]

,published_at,views,likes,comments,category,subscribers,videos_count,source,comments_likes,tags
100,2024-08-25T22:54:22,26097,2134.0,143.0,Разное,NaN,NaN,Rutube,NaN,NaN
869,2024-07-07T16:53:11,11422,963.0,45.0,Разное,NaN,NaN,Rutube,NaN,NaN
995,2024-06-30T18:57:11,12329,941.0,128.0,Разное,NaN,NaN,Rutube,NaN,NaN
1128,2024-06-23T10:00:02,44718,169.0,6.0,Обучение,NaN,NaN,Rutube,NaN,NaN
1465,2024-06-07T14:05:38,102806,660.0,392.0,ПМЭФ,NaN,NaN,Rutube,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
7884,2025-01-27T16:00:59,80398,NaN,NaN,Новости и СМИ,NaN,NaN,Rutube,NaN,NaN
7886,2025-01-26T12:00:29,11702,NaN,NaN,Психология,NaN,NaN,Rutube,NaN,NaN
7887,2025-01-27T10:00:57,64801,NaN,NaN,Развлечения,NaN,NaN,Rutube,NaN,NaN
7888,2025-01-27T18:23:06,46480,NaN,NaN,Телепередачи,NaN,NaN,Rutube,NaN,NaN


In [123]:
df_cleaned[(df_cleaned['subscribers'].isna()) & (df_cleaned['videos_count'].isna())].value_counts('source')

source
Rutube    231
dtype: int64

Как мы видим, там, где нет кол-ва подписчиков, там и нет кол-ва видео на канале. Причем все такие случаи на Rutube.  
Вероятно это каналы, которые были удалены/скрыты и информация о них недоступна, хотя сами видео на площадке остались. Что-ж, такие у них правила.  
Глобально, раз статистика по видео в таких строках доступна, то можем оставить такие строки и заполнить subscribers и videos_count средним значением по Rutube

In [124]:
df_cleaned_rutube = df_cleaned[df_cleaned['source'] == 'Rutube']

df_cleaned['subscribers'] = df_cleaned['subscribers'].fillna(df_cleaned_rutube['subscribers'].mean())
df_cleaned['videos_count'] = df_cleaned['videos_count'].fillna(df_cleaned_rutube['videos_count'].mean())

In [125]:
df_cleaned.isna().sum()

published_at         0
views                0
likes             2139
comments          2035
category             5
subscribers          0
videos_count         0
source               0
comments_likes    6219
tags              7831
dtype: int64

Далее рассмотрим пропуски в лайках и комментариях. Прежде всего увидим, на каких платформах такие пропуски более часты

In [126]:
df_cleaned[df_cleaned['likes'].isna()].value_counts('source')

source
Rutube     2035
Youtube     104
dtype: int64

In [127]:
df_cleaned[df_cleaned['comments'].isna()].value_counts('source')

source
Rutube    2035
dtype: int64

In [128]:
df_cleaned.value_counts('source')

source
Rutube     6219
Youtube    3141
dtype: int64

In [129]:
df_cleaned[df_cleaned['source'] != 'Youtube']['category'].unique()

array(['Авто-мото', 'Новости и СМИ', 'Развлечения', 'Культура', 'Фильмы',
       'Разное', 'Обучение', 'Телепередачи', 'Политика', 'Путешествия',
       'Лайфстайл', 'Интервью', 'Животные', 'Аниме', 'Детям', 'Спорт',
       'Сериалы', 'Юмор', 'Недвижимость', 'Психология', 'Еда',
       'Видеоигры', 'Технологии и интернет',
       'Бизнес и предпринимательство', 'Музыка', 'Наука',
       'Обзоры и распаковки товаров', 'Эзотерика', 'Мультфильмы',
       'Летник RUTUBE', 'Строительство и ремонт', 'Красота', 'Хобби',
       'Техника и оборудование', 'Лайфхаки', 'Здоровье', 'Природа',
       'Гид RUTUBE: города России', 'Сад и огород', 'Спецпроекты', 'ПМЭФ',
       'Дизайн', '80 лет Великой Победе', 'Активность: «Контент-прорыв»',
       'Аудиокниги', 'Акция «Весна на экране»',
       'Акция «RUTUBE — это по любви»', 'Религия',
       'Акция «2024 — встречай!»', 'Включи Новый год'], dtype=object)

In [130]:
df_cleaned[df_cleaned['source'] == 'Youtube']['category'].unique()

array(['Comedy', 'People & Blogs', 'Film & Animation', 'Sports',
       'Science & Technology', 'News & Politics', 'Entertainment',
       'Gaming', 'Music', 'Autos & Vehicles', 'Howto & Style',
       'Education', 'Travel & Events', 'Pets & Animals', nan],
      dtype=object)